In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd

# OpenAIのAPIキーを環境変数から取得
os.environ["OPENAI_API_KEY"] = 'your-api-key'

# モデル名のリスト
model_names = ['gpt-4o-2024-05-13']

# プロンプトテンプレートの読み込み
with open('../subtask_prompt.txt', 'r', encoding='utf-8') as file:
    subtask_prompt_text = file.read()


prompt = PromptTemplate(
    input_variables=["context", "sentence", "subtask_prompt_text"],
    template=(
        "あなたは優秀な医師です。以下の文章全体を考慮し、指定された対象文について、以下のどのラベルに該当するかを判断してください。\n\n"
        "ラベル: \n{subtask_prompt_text}\n\n"
        "文章全体:\n{context}\n\n"
        "対象文:\n{sentence}\n\n"
        "次のトピックに該当する場合は「1」、該当しない場合は「0」としてください。\n"
        "1. omittable\n"
        "2. measure\n"
        "3. extension\n"
        "4. atelectasis\n"
        "5. satellite\n"
        "6. lymphadenopathy\n"
        "7. pleural\n"
        "8. distant\n\n"
        "出力形式: 0または1をスペース区切りで出力してください。\n"
        "例: 1 0 0 0 0 0 0 0\n"
        "出力："
    )
)

# sentences.csv を読み込み
sentences_path = '../radnlp_2024_train_val_20240731/ja/sub_task/val/sentences.csv'
sentences = pd.read_csv(sentences_path)

for model_name in model_names:
    print(f"モデル {model_name} を使用して予測中...")
    output_csv = f'../sentence_predictions_{model_name}.csv'

    # LLMの設定
    if '4o' in model_name:
        llm = ChatOpenAI(temperature=0.7, model_name=model_name)
    else:
        llm = ChatOpenAI(temperature=1, model_name=model_name)

    # チェーンの作成
    chain = LLMChain(llm=llm, prompt=prompt)

    results = []
    
    grouped = sentences.groupby('id')

    for group_id, group in grouped:
        # 文脈としてすべての文を結合
        context = "\n".join(group['text'])
        
        for _, row in group.iterrows():
            sentence = row['text']  # 対象の文
            try:
                # モデルに入力して予測を取得
                output = chain.run({"context": context, "sentence": sentence, "subtask_prompt_text": subtask_prompt_text}).strip()
                # 出力をスペースで分割して数値に変換
                predictions = list(map(int, output.split()))
                # 結果をリストに保存
                results.append({
                    "id": row['id'],
                    "sentence_index": row['sentence_index'],
                    "omittable": predictions[0],
                    "measure": predictions[1],
                    "extension": predictions[2],
                    "atelectasis": predictions[3],
                    "satellite": predictions[4],
                    "lymphadenopathy": predictions[5],
                    "pleural": predictions[6],
                    "distant": predictions[7],
                })
            except Exception as e:
                print(f"エラーが発生しました: ID={row['id']}, sentence_index={row['sentence_index']} - {e}")
                continue

    # DataFrameに変換してCSVファイルに保存
    results_df = pd.DataFrame(results)
    output_csv = '../sentence_classifications.csv'
    results_df.to_csv(output_csv, index=False)

    print(f"結果が{output_csv}に保存されました！")




モデル gpt-4o-2024-05-13 を使用して予測中...
結果が../sentence_classifications.csvに保存されました！
